In [1]:
from os.path import join as pathjoin
import sys
sys.path.insert(0, '/home/mlepekhin/Non-thematic-Text-Classification/code/allennlp_experiments')
from data_processing import *
from interpretation import *
from models import *
from training import *

In [2]:
!gpustat

seven                Tue Jan 31 20:03:30 2023  470.161.03
[0] NVIDIA TITAN RTX | 26'C,   0 % |  2939 / 24220 MB | mlepekhin(1091M) yadimasek(1845M)
[1] NVIDIA TITAN RTX | 27'C,   0 % |     3 / 24220 MB |
[2] NVIDIA TITAN RTX | 30'C,   0 % |     3 / 24220 MB |
[3] NVIDIA TITAN RTX | 28'C,   0 % |     3 / 24220 MB |


Let's download all the data.

In [3]:
!mkdir /home/mlepekhin/models/competition


DATA_DIR = '/home/mlepekhin/data'
MODELS_DIR = '/home/mlepekhin/models/competition'
MODEL_ID = 'subtask2_bert_fr_multi' 
CHECKPOINTS_DIR = pathjoin(MODELS_DIR, MODEL_ID, 'checkpoints')
BEST_MODEL = pathjoin(CHECKPOINTS_DIR, 'best.th')

mkdir: cannot create directory ‘/home/mlepekhin/models/competition’: File exists


In [4]:
transformer_model = 'bert-base-multilingual-cased'
MAX_TOKENS = 512

In [5]:
train_dataset_reader = build_multilabel_transformer_dataset_reader(transformer_model, lower=True)
val_dataset_reader = build_multilabel_transformer_dataset_reader(transformer_model, lower=True)

train_data, dev_data = read_data(
    "subtask2_fr_train.csv", 
    "subtask2_fr_train.csv", 
    train_dataset_reader, 
    val_dataset_reader
)

vocab = build_vocab(train_data + dev_data)

train_data.index_with(vocab)
dev_data.index_with(vocab)

01312023 20:03:31|INFO|transformers.configuration_utils| loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-config.json from cache at /home/mlepekhin/.cache/torch/transformers/45629519f3117b89d89fd9c740073d8e4c1f0a70f9842476185100a8afe715d1.65df3cef028a0c91a7b059e4c404a975ebe6843c71267b67019c0e9cfa8a88f0
01312023 20:03:31|INFO|transformers.configuration_utils| Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "firs

01312023 20:03:38|INFO|transformers.configuration_utils| loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-config.json from cache at /home/mlepekhin/.cache/torch/transformers/45629519f3117b89d89fd9c740073d8e4c1f0a70f9842476185100a8afe715d1.65df3cef028a0c91a7b059e4c404a975ebe6843c71267b67019c0e9cfa8a88f0
01312023 20:03:38|INFO|transformers.configuration_utils| Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "firs

Reading data
<class 'data_processing.MultiLabelClassificationDatasetReader'> subtask2_fr_train.csv


0it [00:00, ?it/s]

0it [00:00, ?it/s]

01312023 20:03:45|INFO|allennlp.data.vocabulary| Fitting token dictionary from dataset.


Building the vocabulary


  0%|          | 0/316 [00:00<?, ?it/s]

In [6]:
model = build_multilabel_transformer_model(vocab, transformer_model)

if torch.cuda.is_available():
    cuda_device = 2
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
print(cuda_device)

model.load_state_dict(torch.load(BEST_MODEL, map_location=f'cuda:{cuda_device}'))

Building the model


01312023 20:03:46|INFO|transformers.configuration_utils| loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-config.json from cache at /home/mlepekhin/.cache/torch/transformers/45629519f3117b89d89fd9c740073d8e4c1f0a70f9842476185100a8afe715d1.65df3cef028a0c91a7b059e4c404a975ebe6843c71267b67019c0e9cfa8a88f0
01312023 20:03:46|INFO|transformers.configuration_utils| Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "firs

2


<All keys matched successfully>

In [7]:
#!rm -rf {CHECKPOINTS_DIR}
#!mkdir -p {CHECKPOINTS_DIR}

In [8]:
'''train_loader, dev_loader = build_data_loaders(train_data, dev_data)

# You obviously won't want to create a temporary file for your training
# results, but for execution in binder for this course, we need to do this.

trainer = build_classifier_trainer(
    model,
    pathjoin(MODELS_DIR, MODEL_ID, 'checkpoints'),
    train_loader,
    dev_loader,
    2,
    cuda_device=cuda_device
)
print("Starting training")
trainer.train()
print("Finished training")'''

'train_loader, dev_loader = build_data_loaders(train_data, dev_data)\n\n# You obviously won\'t want to create a temporary file for your training\n# results, but for execution in binder for this course, we need to do this.\n\ntrainer = build_classifier_trainer(\n    model,\n    pathjoin(MODELS_DIR, MODEL_ID, \'checkpoints\'),\n    train_loader,\n    dev_loader,\n    2,\n    cuda_device=cuda_device\n)\nprint("Starting training")\ntrainer.train()\nprint("Finished training")'

In [9]:
#!rm "{CHECKPOINTS_DIR}"/*.json "{CHECKPOINTS_DIR}"/model* "{CHECKPOINTS_DIR}"/training*
#model.vocab.save_to_files(pathjoin(MODELS_DIR, MODEL_ID, 'vocab'))
#!ls -lh "{CHECKPOINTS_DIR}"

In [10]:
from evaluate_transformer_classifier import *

In [11]:
dev_text_list = pd.read_csv('subtask2_fr_dev.csv').text.values
#targets = pd.read_csv('subtask2_fr_dev.csv').target.values
tokenizer = PretrainedTransformerTokenizer(transformer_model)
indexer = PretrainedTransformerIndexer(transformer_model)
    
    
probs_list = predict_probs_for_texts(dev_text_list, 8, model, tokenizer, indexer, vocab, device=f'cuda:{cuda_device}')

01312023 20:04:01|INFO|transformers.configuration_utils| loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-config.json from cache at /home/mlepekhin/.cache/torch/transformers/45629519f3117b89d89fd9c740073d8e4c1f0a70f9842476185100a8afe715d1.65df3cef028a0c91a7b059e4c404a975ebe6843c71267b67019c0e9cfa8a88f0
01312023 20:04:01|INFO|transformers.configuration_utils| Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "firs

In [12]:
label_list = list(vocab.get_index_to_token_vocabulary('labels').values())


In [19]:
dev_ids = pd.read_csv('subtask2_fr_dev.csv').article_id.values

with open('results_2/subtask2_fr_dev_bert_multi_threshold_0.3.txt', 'w') as fout:
    for dev_id, probs in zip(dev_ids, probs_list):
        fout.write(f'{dev_id}\t')
        
        was = False
        for label, prob in zip(label_list, probs):
            if prob >= 0.3:
                if not was:
                    was = True
                else:
                    fout.write(',')
                fout.write(label)
        fout.write('\n')

In [20]:
probs

array([0.4164196 , 0.3998041 , 0.46029422, 0.38465193, 0.27300185,
       0.2773036 , 0.24660902, 0.22796872, 0.25650993, 0.18383043,
       0.1868735 , 0.21030937, 0.2096996 , 0.17918247], dtype=float32)

In [21]:
dev_ids = pd.read_csv('subtask2_fr_test.csv').article_id.values

with open('results_2/subtask2_fr_test_bert_multi_threshold_0.3.txt', 'w') as fout:
    for dev_id, probs in zip(dev_ids, probs_list):
        fout.write(f'{dev_id}\t')
        
        was = False
        for label, prob in zip(label_list, probs):
            if prob >= 0.3:
                if not was:
                    was = True
                else:
                    fout.write(',')
                fout.write(label)
        fout.write('\n')